# To run this file, install Python packages: 
- requests
- tabulate

In [9]:
import json
import requests
from random import randrange
from tabulate import tabulate

# Year range
start_year = 2010
end_year = 2020
year = randrange(start_year, end_year)
game_season = str(year) + "-" + str((year + 1) % 100).rjust(2, "0")

# Game type
game_types = ["Regular Season", "Playoffs"]
game_type = game_types[randrange(len(game_types))]

# Game data
players_url = "https://api.pbpstats.com/get-games/nba"
games_params = {
    "Season": game_season,
    "SeasonType": game_type
}
response = requests.get(players_url, params=games_params)
response_json = response.json()["results"]

# Random a game
game_index = randrange(len(response_json))
game_overview = response_json[game_index]
print(json.dumps(game_overview, indent=2))

# Get game stats from the overview
print("Extracting game stats for game:", game_overview["GameId"], "...")
players_url = "https://api.pbpstats.com/get-game-stats"
games_params = {
    "GameId": game_overview["GameId"],
    "Type": "Player" # options are Player, Lineup and LineupOpponent
}
response = requests.get(players_url, params=games_params)
game_stats = response.json()['stats']
print("Done")

{
  "GameId": "0041700233",
  "Date": "2018-05-04",
  "HomeTeamId": "1610612740",
  "AwayTeamId": "1610612744",
  "HomePoints": 119,
  "AwayPoints": 100,
  "HomePossessions": 103,
  "AwayPossessions": 102,
  "HomeTeamAbbreviation": "NOP",
  "AwayTeamAbbreviation": "GSW"
}
Extracting game stats for game: 0041700233 ...
Done


## Some util functions:

In [10]:
# Stats to get
key_general = ["GamesPlayed"]
key_attack_stats = ["Fg2Pct", "Fg3Pct", "FtPoints", "FTA", "Turnovers"]
key_defend_stats = ["Steals", "Blocks", "Fouls"]
key_all_stats = key_general + key_attack_stats + key_defend_stats


# Uitil functions
def get_team_players(game_stats, team_type):
    team = []
    if team_type == "Away":
        stats = game_stats["Away"]["FullGame"]
    elif team_type == "Home":
        stats = game_stats["Home"]["FullGame"]

    for entity in stats:
        if entity["Name"] != "Team":
            team.append([entity["Name"], entity["Minutes"]])
    team.sort(key=lambda lst: lst[1], reverse=True)
    return team


def get_player_ids(team, team_type):
    if team_type == "Away":
        team_id = game_overview["AwayTeamId"]
    elif team_type == "Home":
        team_id = game_overview["HomeTeamId"]

    players_url = "https://api.pbpstats.com/get-team-players-for-season"
    players_params = {
        "Season": game_season,
        "SeasonType": game_type,
        "TeamId": team_id
    }
    players_response = requests.get(players_url, params=players_params)
    players = players_response.json()["players"]

    # "team" is list of list, each sub list starts with player name
    for player in team:
        for id, name in players.items():
            if name == player[0]:
                player.append(id)


def get_player_stats_by_id(id):
    url = "https://api.pbpstats.com/get-all-season-stats/nba"
    params = {
        "EntityType": "Player",
        "EntityId": id
    }
    player_stats = requests.get(url, params=params).json()

    for json_object in player_stats["results"][game_type]:
        for key, value in json_object.items():
            if key == "Season" and value == game_season:
                return json_object


def get_team_stats(team):
    for player in team:
        all_stats = get_player_stats_by_id(player[2])
        for key in key_all_stats:
            if key in all_stats:
                player.append(all_stats[key])
            else:
                player.append(None)


# Print messages
col_names = ["Name", "Minutes", "ID"] + key_all_stats
def print_table(team):
    print(tabulate(team, headers=col_names, tablefmt="grid"))

## Away team stats:

In [11]:
# Away team stats
print("Away team players:")
away_team = get_team_players(game_stats, "Away")
get_player_ids(away_team, "Away")
get_team_stats(away_team)
print_table(away_team)

Away team players:
+------------------+-----------+---------+---------------+----------+----------+------------+-------+-------------+----------+----------+---------+
| Name             | Minutes   |      ID |   GamesPlayed |   Fg2Pct |   Fg3Pct |   FtPoints |   FTA |   Turnovers |   Steals |   Blocks |   Fouls |
+==================+===========+=========+===============+==========+==========+============+=======+=============+==========+==========+=========+
| Draymond Green   | 36:25     |  203110 |            21 | 0.548673 | 0.265823 |         39 |    49 |          63 |       41 |       31 |      75 |
+------------------+-----------+---------+---------------+----------+----------+------------+-------+-------------+----------+----------+---------+
| Klay Thompson    | 36:22     |  202691 |            21 | 0.497297 | 0.426752 |         27 |    31 |          31 |       16 |        6 |      48 |
+------------------+-----------+---------+---------------+----------+----------+------------+

## Home team stats:

In [12]:
# Home team stats
print("Home team players:")
home_team = get_team_players(game_stats, "Home")
get_player_ids(home_team, "Home")
get_team_stats(home_team)
print_table(home_team)

Home team players:
+-----------------+-----------+---------+---------------+----------+----------+------------+-------+-------------+----------+----------+---------+
| Name            | Minutes   |      ID |   GamesPlayed |   Fg2Pct |   Fg3Pct |   FtPoints |   FTA |   Turnovers |   Steals |   Blocks |   Fouls |
+=================+===========+=========+===============+==========+==========+============+=======+=============+==========+==========+=========+
| Anthony Davis   | 40:35     |  203076 |             9 | 0.549451 | 0.272727 |         53 |    64 |          23 |       18 |       21 |      30 |
+-----------------+-----------+---------+---------------+----------+----------+------------+-------+-------------+----------+----------+---------+
| Rajon Rondo     | 37:14     |  200765 |             9 | 0.410959 | 0.421053 |          9 |    14 |          33 |       13 |        2 |      20 |
+-----------------+-----------+---------+---------------+----------+----------+------------+-------